In [1]:
import pyvista as pv
import meshio
import SimpleITK as sitk
import vtk
import numpy as np
from tqdm import tqdm
from pathlib import Path
from meshpy.tet import MeshInfo, build
from scipy.ndimage import label

## Functions

In [11]:
def hex_to_5tets(hexes):
    a,b,c,d,e,f,g,h = hexes[:,0],hexes[:,1],hexes[:,2],hexes[:,3], \
                      hexes[:,4],hexes[:,5],hexes[:,6],hexes[:,7]
    return np.vstack([
        np.stack([a,b,d,e],axis=1),
        np.stack([b,c,d,g],axis=1),
        np.stack([b,e,f,g],axis=1),
        np.stack([b,d,e,g],axis=1),
        np.stack([d,e,g,h],axis=1),
    ])

def numpy_to_vtk_coords(coords):
    vtkarr = vtk.vtkDoubleArray()
    vtkarr.SetNumberOfValues(len(coords))
    for i, v in enumerate(coords):
        vtkarr.SetValue(i, float(v))
    return vtkarr

def replace_header_and_elements(inp_file, new_header_file, output_file):
    # Read new header
    with open(new_header_file, "r") as f:
        new_header = f.read().rstrip("\n") + "\n"

    lines = []
    header_inserted = False

    with open(inp_file, "r") as f:
        for line in f:
            stripped = line.strip()

            # --- 1. Replace header before *NODE
            if not header_inserted and stripped.upper().startswith("*NODE"):
                lines.append(new_header)
                header_inserted = True
                # skip writing the *NODE line itself
                continue

            # --- 2. Modify *ELEMENT, TYPE= lines
            if stripped.upper().startswith("*ELEMENT, TYPE="):
                parts = line.split("=", 1)
                if len(parts) == 2:
                    elem_type = parts[1].strip().upper()
                    line = f"*Element, type={elem_type}\n"

            # Add the processed line
            lines.append(line)

    # Write result
    with open(output_file, "w") as f:
        f.writelines(lines)

def append_footer(inp_file, footer_file, output_file):
    # Read footer
    with open(footer_file, "r") as f:
        footer = f.read()

    with open(inp_file, "r") as f:
        content = f.read()

    with open(output_file, "w") as f:
        f.write(content.rstrip("\n") + "\n")  # ensure last line ends
        f.write(footer.rstrip("\n") + "\n")

#Clean mask from small voxels
def remove_small_islands(binary_matrix, area_threshold):
    """Remove small connected components from a binary mask."""
    labeled_array, num_features = label(binary_matrix)
    for i in range(1, num_features + 1):
        component = (labeled_array == i)
        if component.sum() < area_threshold:
            binary_matrix[component] = 0
    return binary_matrix

def mask_region_of_interest(mask,ranges):
    zero_mask=np.zeros(np.shape(mask))
    zero_mask[ranges[0]:ranges[1],ranges[2]:ranges[3],ranges[4]:ranges[5]]=mask[ranges[0]:ranges[1],ranges[2]:ranges[3],ranges[4]:ranges[5]]
    return zero_mask

def create_blocks(mask, thickness, axis):
    if (axis=="X"):
        xmin=np.min(np.where(mask==1)[0])
        xmax=np.max(np.where(mask==1)[0])+1
        ymin=np.min([np.min(np.where(mask[xmin:xmin+thickness,:,:]==1)[1]),np.min(np.where(mask[xmax-thickness:xmax,:,:]==1)[1])])
        ymax=np.max([np.max(np.where(mask[xmin:xmin+thickness,:,:]==1)[1]),np.max(np.where(mask[xmax-thickness:xmax,:,:]==1)[1])])+1
        zmin=np.min([np.min(np.where(mask[xmin:xmin+thickness,:,:]==1)[2]),np.min(np.where(mask[xmax-thickness:xmax,:,:]==1)[2])])
        zmax=np.max([np.max(np.where(mask[xmin:xmin+thickness,:,:]==1)[2]),np.max(np.where(mask[xmax-thickness:xmax,:,:]==1)[2])])+1
        blocks=np.zeros(np.shape(mask))
        blocks[xmin:xmin+thickness,ymin:ymax,zmin:zmax]=2
        blocks[xmax-thickness:xmax,ymin:ymax,zmin:zmax]=2
        final_mask=blocks+mask
        return final_mask

## Import the .nddr CT file

In [34]:
import os
import numpy as np
import SimpleITK as sitk
import pyvista as pv
from meshpy.tet import MeshInfo, build
from pathlib import Path
from scipy.ndimage import map_coordinates
import meshlib.mrmeshpy as mr
import meshlib.mrmeshnumpy as mrn
import random

# ------------------------
# PARAMETERS
# ------------------------
input_nrrd = "microCT_volume_preview.nrrd"
iso_value = 35000         # Threshold for segmentation
downsample_factor = 4     # Reduce voxel count for speed (optional)

# ------------------------
# 1. LOAD IMAGE
# ------------------------
if not os.path.exists(input_nrrd):
    raise FileNotFoundError(f"File not found: {input_nrrd}")

image = sitk.ReadImage(input_nrrd)
orig_size = np.array(image.GetSize())
orig_spacing = np.array(image.GetSpacing())
origin = np.array(image.GetOrigin())

# ------------------------
# 2. DOWNSAMPLE (OPTIONAL)
# ------------------------
new_size = (orig_size / downsample_factor).astype(int)
new_spacing = orig_spacing * (orig_size / new_size)

resampler = sitk.ResampleImageFilter()
resampler.SetSize([int(s) for s in new_size])
resampler.SetOutputSpacing([float(s) for s in new_spacing])
resampler.SetOutputOrigin(origin)
resampler.SetOutputDirection(image.GetDirection())
resampler.SetInterpolator(sitk.sitkLinear)
image = resampler.Execute(image)

# Convert to numpy array
array = sitk.GetArrayFromImage(image)  # z, y, x
spacing = np.array(image.GetSpacing())
origin = np.array(image.GetOrigin())
print(f"Loaded image: shape={array.shape}, intensity range=({array.min()}, {array.max()})")

# ------------------------
# 3. APPLY THRESHOLD
# ------------------------
mask = array >= iso_value
n_voxels = np.sum(mask)
print(f"Threshold applied: {n_voxels} voxels above threshold")
if n_voxels == 0:
    raise ValueError("No voxels above threshold. Reduce iso_value.")

mask = remove_small_islands(mask, 30)
mask = mask_region_of_interest(mask,[5,70,5,70,5,100])
mask = create_blocks(mask,5,"X")

# Generate STL surface
simpleVolume = mrn.simpleVolumeFrom3Darray(np.float32(mask>0))
floatGrid = mr.simpleVolumeToDenseGrid(simpleVolume)
mesh_stl = mr.gridToMesh(floatGrid, mr.Vector3f(1.0, 1.0, 1.0), 0.5)
stl_path = Path(input_nrrd).stem + "_TETmesh.stl"
mr.saveMesh(mesh_stl, stl_path)

mesh_nuclei = pv.read(stl_path)
if mesh_nuclei.volume > 0.0:
    mesh_nuclei.decimate(target_reduction=0.8, inplace=True)

# ------------------------
# 4. TETRAHEDRALIZATION
# ------------------------
surface_mesh = pv.read(stl_path)
points = np.array(surface_mesh.points)
faces = surface_mesh.faces.reshape(-1, 4)[:, 1:]

mesh_info = MeshInfo()
mesh_info.set_points(points)
mesh_info.set_facets(faces.tolist())

tet_mesh = build(mesh_info, max_volume=1.0)

tet_points = np.array(tet_mesh.points)
tet_elements = []
for tet in tet_mesh.elements:
    tet_elements.extend([4, *tet])  # 4 nodes per tetra
tet_elements = np.array(tet_elements)
celltypes = np.full(len(tet_mesh.elements), pv.CellType.TETRA, dtype=np.uint8)

grid = pv.UnstructuredGrid(tet_elements, celltypes, tet_points)

# ------------------------
# DEBUGGING COORDINATES
# ------------------------
print("\n--- Coordinate Sanity Check ---")
print("Voxel index limits (z,y,x):", array.shape)
print("Centroid range:", tet_points.min(axis=0), tet_points.max(axis=0))

# ------------------------
# 6. DENSITY & YOUNG MODULUS SAMPLING
# ------------------------
cell_densities = []
cell_ymodulus = []
for tet in tet_mesh.elements:
    centroid = tet_points[tet].mean(axis=0)
    v_coord = centroid[[0, 1, 2]]
    if mask[round(v_coord[0]), round(v_coord[1]), round(v_coord[2])]<2:
        density_val = map_coordinates(array, v_coord.reshape(3, 1), order=1, mode='nearest')
        #Conversion
        density_val=(density_val[0]*3.85)/6.6e4
        ymodulus_val=(density_val**2.0)*1.4e4
        cell_densities.append(density_val)
        cell_ymodulus.append(ymodulus_val)
    else:
        cell_densities.append(5.0)
        cell_ymodulus.append(100000)

grid.cell_data["Density (g/cm3)"] = np.array(cell_densities)
grid.cell_data["YM (MPa)"] = np.array(cell_ymodulus)

# ------------------------
# 6. SAVE VTK
# ------------------------
vtk_path = Path(input_nrrd).stem + "_TETmesh.vtk"
grid.save(vtk_path)
print(f"Tetrahedral mesh with densities saved to {vtk_path}")

Loaded image: shape=(149, 128, 119), intensity range=(0, 65535)
Threshold applied: 62139 voxels above threshold

--- Coordinate Sanity Check ---
Voxel index limits (z,y,x): (149, 128, 119)
Centroid range: [ 4.5 24.5 32.5] [69.5 69.5 98.5]
Tetrahedral mesh with densities saved to microCT_volume_preview_TETmesh.vtk


In [35]:
from collections import defaultdict

tolerance = 0.01  # 1% difference allowed
sorted_elements = sorted(
    [(i + 1, E) for i, E in enumerate(cell_ymodulus)],
    key=lambda x: x[1]
)

E_groups = []
group_values = []

for elem_id, E in sorted_elements:
    placed = False
    for idx, g_val in enumerate(group_values):
        if abs(E - g_val) / g_val <= tolerance:
            E_groups[idx].append(elem_id)
            # update group value as average to keep cluster centered
            group_values[idx] = (group_values[idx] * (len(E_groups[idx]) - 1) + E) / len(E_groups[idx])
            placed = True
            break
    if not placed:
        group_values.append(E)
        E_groups.append([elem_id])

In [36]:
# Write INP
inp_path = Path(input_nrrd).stem + "_TETmesh_binned_E.inp"
with open(inp_path, "w") as f:
    f.write("*Heading\n")
    f.write("** Generated by Python script\n")
    
    # Nodes
    f.write("*Node\n")
    for i, p in enumerate(tet_points, start=1):
        f.write(f"{i}, {p[0]:.6f}, {p[1]:.6f}, {p[2]:.6f}\n")
    
    # Elements
    f.write("*Element, type=C3D4\n")
    for i, e in enumerate(tet_mesh.elements, start=1):
        f.write(f"{i}, {e[0]+1}, {e[1]+1}, {e[2]+1}, {e[3]+1}\n")

    # Materials and sections per group
    for mat_idx, (E_val, elems) in enumerate(zip(group_values, E_groups), start=1):
        f.write(f"*Elset, elset=ESET{mat_idx}\n")
        for e_id in elems:
            f.write(f"{e_id},\n")
        
        f.write(f"*Material, name=MAT{mat_idx}\n")
        f.write("*Elastic\n")
        f.write(f"{E_val:.6f}, 0.3\n")  # Poisson's ratio constant
        
        f.write(f"*Solid Section, elset=ESET{mat_idx}, material=MAT{mat_idx}\n")

## Generate HEX and TET meshes

In [4]:
# Load the rectilinear grid from VTK
vtk_file=Path(input_nrrd).stem + "_TETmesh.vtk"
inp_file=Path(input_nrrd).stem + "_TETmesh.inp"
mesh = pv.read(vtk_file)
np.unique(mesh.celltypes)

# Nodes
points = mesh.points

# Elements
cells = mesh.cells.reshape(-1, 5)[:, 1:]  # skip the '4' prefix for each tetra

# Write INP file
with open(inp_file, "w") as f:
    f.write("*Heading\n")
    f.write("** Generated from VTK\n")
    f.write("*NODE\n")
    for i, p in enumerate(points, start=1):
        f.write(f"{i}, {p[0]}, {p[1]}, {p[2]}\n")
    
    f.write("*ELEMENT, TYPE=C3D4\n")
    for i, e in enumerate(cells, start=1):
        # Abaqus uses 1-based indexing
        f.write(f"{i}, {e[0]+1}, {e[1]+1}, {e[2]+1}, {e[3]+1}\n")
        
replace_header_and_elements(Path(input_nrrd).stem + "_TETmesh.inp", "inp_header.txt", Path(input_nrrd).stem + "_TETmesh.inp")
append_footer(Path(input_nrrd).stem + "_TETmesh.inp", "inp_footer.txt", Path(input_nrrd).stem + "_TETmesh.inp")

print(f"INP file saved to {inp_file}")

INP file saved to microCT_volume_preview_TETmesh.inp


In [5]:
# ------------------------
# 7. EXPORT TO ABAQUS INP
# ------------------------
inp_path = Path(input_nrrd).stem + "_TETmesh_with_E.inp"
with open(inp_path, "w") as f:
    # Heading
    f.write("*Heading\n")
    f.write("** Generated by Python script\n")
    
    # Nodes
    f.write("*Node\n")
    for i, p in enumerate(tet_points, start=1):
        f.write(f"{i}, {p[0]:.6f}, {p[1]:.6f}, {p[2]:.6f}\n")
    
    # Elements
    f.write("*Element, type=C3D4\n")
    for i, e in enumerate(tet_mesh.elements, start=1):
        f.write(f"{i}, {e[0]+1}, {e[1]+1}, {e[2]+1}, {e[3]+1}\n")

    # Material definition for each element (via distribution)
    f.write("*Solid Section, material=Bone, elset=AllElements\n")
    f.write("*Elset, elset=AllElements, generate\n")
    f.write(f"1, {len(tet_mesh.elements)}, 1\n")
    f.write("*Distribution, name=YoungsDist, location=element\n")
    for i, E in enumerate(cell_ymodulus, start=1):
        f.write(f"{i}, {E:.6f}\n")
    
    f.write("*Material, name=Bone\n")
    f.write("*Elastic, dependencies=1\n")
    f.write("1., 0.3\n")  # placeholder, distribution overrides E

#### Extract subset?

In [4]:
import vtk

# --- Read rectilinear grid ---
reader = vtk.vtkRectilinearGridReader()
reader.SetFileName(Path(input_file).stem + ".vtk")
reader.Update()

rgrid = reader.GetOutput()

# --- Get max indices along each axis ---
dims = rgrid.GetDimensions()  # (nx, ny, nz) = number of points in each direction
xmax = dims[0] - 1
ymax = dims[1] - 1
zmax = dims[2] - 1

print(f"Max index X: {xmax}, Y: {ymax}, Z: {zmax}")

# --- Extract subset (VOI = Volume of Interest) ---
extract = vtk.vtkExtractRectilinearGrid()
extract.SetInputData(rgrid)

# Example: keep voxels from (x: 10–50, y: 20–80, z: 5–40)
extract.SetVOI(0, xmax,   # x-min, x-max (indices)
               0, 150,   # y-min, y-max
               0, 40)    # z-min, z-max

extract.Update()
subgrid = extract.GetOutput()

# --- Write subset to file ---
writer = vtk.vtkRectilinearGridWriter()
writer.SetFileName(Path(input_file).stem + ".vtk")
writer.SetInputData(subgrid)
writer.Write()

print(f"Subset saved as {Path(input_file).stem}.vtk")

Max index X: 238, Y: 256, Z: 298
Subset saved as microCT_volume_preview.vtk
